In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya_speech.train.model.conformer as conformer
import malaya_speech.train.model.transducer as transducer
import malaya_speech
import tensorflow as tf
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
subwords = malaya_speech.subword.load('transducer.subword')

In [4]:
featurizer = malaya_speech.tf_featurization.STTFeaturizer(
    normalize_per_feature = True
)

In [5]:
X = tf.compat.v1.placeholder(tf.float32, [None, None], name = 'X_placeholder')
X_len = tf.compat.v1.placeholder(tf.int32, [None], name = 'X_len_placeholder')

In [6]:
batch_size = tf.shape(X)[0]
features = tf.TensorArray(dtype = tf.float32, size = batch_size, dynamic_size = True, infer_shape = False)
features_len = tf.TensorArray(dtype = tf.int32, size = batch_size)

init_state = (0, features, features_len)

def condition(i, features, features_len):
    return i < batch_size

def body(i, features, features_len):
    f = featurizer(X[i, :X_len[i]])
    f_len = tf.shape(f)[0]
    return i + 1, features.write(i, f), features_len.write(i, f_len)

_, features, features_len = tf.while_loop(condition, body, init_state)
features_len = features_len.stack()
padded_features = tf.TensorArray(dtype = tf.float32, size = batch_size)
padded_lens = tf.TensorArray(dtype = tf.int32, size = batch_size)
maxlen = tf.reduce_max(features_len)

init_state = (0, padded_features, padded_lens)

def condition(i, padded_features, padded_lens):
    return i < batch_size

def body(i, padded_features, padded_lens):
    f = features.read(i)
    len_f = tf.shape(f)[0]
    f = tf.pad(f, [[0, maxlen - tf.shape(f)[0]], [0,0]])
    return i + 1, padded_features.write(i, f), padded_lens.write(i, len_f)

_, padded_features, padded_lens = tf.while_loop(condition, body, init_state)
padded_features = padded_features.stack()
padded_lens = padded_lens.stack()
padded_lens.set_shape((None))
padded_features.set_shape((None, None, 80))
padded_features = tf.expand_dims(padded_features, -1)
padded_features, padded_lens

(<tf.Tensor 'ExpandDims:0' shape=(?, ?, 80, 1) dtype=float32>,
 <tf.Tensor 'TensorArrayStack_2/TensorArrayGatherV3:0' shape=(?,) dtype=int32>)

In [7]:
padded_features = tf.identity(padded_features, name = 'padded_features')
padded_lens = tf.identity(padded_lens, name = 'padded_lens')

In [8]:
config = malaya_speech.config.conformer_large_encoder_config
config['dropout'] = 0.0
conformer_model = conformer.Model(**config)
decoder_config = malaya_speech.config.conformer_large_decoder_config
decoder_config['embed_dropout'] = 0.0
transducer_model = transducer.rnn.Model(
    conformer_model, vocabulary_size = subwords.vocab_size, **decoder_config
)

In [9]:
p = tf.compat.v1.placeholder(tf.int32, [None, None])
z = tf.zeros((tf.shape(p)[0], 1),dtype=tf.int32)
c = tf.concat([z, p], axis = 1)
p_len = tf.compat.v1.placeholder(tf.int32, [None])
c

<tf.Tensor 'concat:0' shape=(?, ?) dtype=int32>

In [10]:
training = True

In [11]:
logits = transducer_model([padded_features, c, p_len], training = training)
logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<tf.Tensor 'transducer/transducer_joint/transducer_joint_vocab/BiasAdd:0' shape=(?, ?, ?, 1030) dtype=float32>

In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [13]:
var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
saver = tf.train.Saver(var_list = var_list)
saver.restore(sess, 'asr-large-conformer-transducer/model.ckpt-400000')

INFO:tensorflow:Restoring parameters from asr-large-conformer-transducer/model.ckpt-400000


In [14]:
decoded = transducer_model.greedy_decoder(padded_features, padded_lens, training = training)
decoded = tf.identity(decoded, name = 'greedy_decoder')
decoded

<tf.Tensor 'greedy_decoder:0' shape=(?, ?) dtype=int32>

In [15]:
encoded = transducer_model.encoder(padded_features, training = training)
encoded = tf.identity(encoded, name = 'encoded')
encoded_placeholder = tf.placeholder(tf.float32, [config['dmodel']], name = 'encoded_placeholder')
predicted_placeholder = tf.placeholder(tf.int32, None, name = 'predicted_placeholder')
t = transducer_model.predict_net.get_initial_state().shape
states_placeholder = tf.placeholder(tf.float32, [int(i) for i in t], name = 'states_placeholder')

ytu, new_states = transducer_model.decoder_inference(
    encoded=encoded_placeholder,
    predicted=predicted_placeholder,
    states=states_placeholder,
    training = training
)

ytu = tf.identity(ytu, name = 'ytu')
new_states = tf.identity(new_states, name = 'new_states')
ytu, new_states

(<tf.Tensor 'ytu:0' shape=(1030,) dtype=float32>,
 <tf.Tensor 'new_states:0' shape=(1, 2, 1, 640) dtype=float32>)

In [16]:
initial_states = transducer_model.predict_net.get_initial_state()
initial_states = tf.identity(initial_states, name = 'initial_states')

In [17]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

In [18]:
# var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
# saver = tf.train.Saver(var_list = var_list)
# saver.restore(sess, 'asr-small-conformer-transducer/model.ckpt-325000')

In [19]:
files = [
    'speech/record/savewav_2020-11-26_22-36-06_294832.wav',
    'speech/record/savewav_2020-11-26_22-40-56_929661.wav',
    'speech/record/675.wav',
    'speech/record/664.wav',
    'speech/example-speaker/husein-zolkepli.wav',
    'speech/example-speaker/mas-aisyah.wav',
    'speech/example-speaker/khalil-nooh.wav',
    'speech/example-speaker/shafiqah-idayu.wav',
    'speech/khutbah/wadi-annuar.wav',
]

ys = [malaya_speech.load(f)[0] for f in files]
padded, lens = malaya_speech.padding.sequence_1d(ys, return_len = True)

In [20]:
import collections
import numpy as np
import tensorflow as tf

BeamHypothesis = collections.namedtuple(
    'BeamHypothesis', ('score', 'prediction', 'states')
)


def transducer(
    enc,
    total,
    initial_states,
    encoded_placeholder,
    predicted_placeholder,
    states_placeholder,
    ytu,
    new_states,
    sess,
    beam_width = 10,
    norm_score = True,
):
    kept_hyps = [
        BeamHypothesis(score = 0.0, prediction = [0], states = initial_states)
    ]
    B = kept_hyps
    for i in range(total):
        A = B
        B = []
        while True:
            y_hat = max(A, key = lambda x: x.score)
            A.remove(y_hat)
            ytu_, new_states_ = sess.run(
                [ytu, new_states],
                feed_dict = {
                    encoded_placeholder: enc[i],
                    predicted_placeholder: y_hat.prediction[-1],
                    states_placeholder: y_hat.states,
                },
            )
            for k in range(ytu_.shape[0]):
                beam_hyp = BeamHypothesis(
                    score = (y_hat.score + float(ytu_[k])),
                    prediction = y_hat.prediction,
                    states = y_hat.states,
                )
                if k == 0:
                    B.append(beam_hyp)
                else:
                    beam_hyp = BeamHypothesis(
                        score = beam_hyp.score,
                        prediction = (beam_hyp.prediction + [int(k)]),
                        states = new_states_,
                    )
                    A.append(beam_hyp)
            if len(B) > beam_width:
                break
    if norm_score:
        kept_hyps = sorted(
            B, key = lambda x: x.score / len(x.prediction), reverse = True
        )[:beam_width]
    else:
        kept_hyps = sorted(B, key = lambda x: x.score, reverse = True)[
            :beam_width
        ]
    return kept_hyps[0].prediction

In [21]:
%%time

r = sess.run(decoded, feed_dict = {X: padded, X_len: lens})
for row in r:
    print(malaya_speech.subword.decode(subwords, row[row > 0]))

helo nama saya sen saya tak suka mandi kejap saya masam
helo nama saya using saya suka mandi saya mandi titia
ini dan melalui kenyataan mesej itu mastura menegaskan
pilihan tepat apabila dia kini lebih berani dan
testing nama saya using bin zulkifli
sebut perkataan uncle
tolong sebut nanti kata
nama saya syafiqah idayu
jadi dalam perjalanan ini dunia yang susah ini ketika nabi mengajar muaz bin jabal tadi ni alah ni
CPU times: user 58.7 s, sys: 5.66 s, total: 1min 4s
Wall time: 13.4 s


In [22]:
%%time

encoded_, padded_lens_  = sess.run([encoded, padded_lens], feed_dict = {X: padded, X_len: lens})
padded_lens_ = padded_lens_ // conformer_model.conv_subsampling.time_reduction_factor
s = sess.run(initial_states)

for i in range(len(encoded_)):
    r = transducer(
        enc = encoded_[i],
        total = padded_lens_[i],
        initial_states = s,
        encoded_placeholder = encoded_placeholder,
        predicted_placeholder = predicted_placeholder,
        states_placeholder = states_placeholder,
        ytu = ytu,
        new_states = new_states,
        sess = sess,
        beam_width = 1,
    )

    print(malaya_speech.subword.decode(subwords, r))

helo nama saya sen saya tak suka mandi kejap saya masam
helo nama saya using saya suka mandi saya mandi titiai
ini dan melalui kenyataan mesej itu mastura menegaskan
pilihan tepat apabila dia kini lebih berani dan
testing nama saya using bin zulkifli
sebut perkataan uncle
tolong sebut nanti kata
nama saya syafiqah idayu
jadi dalam perjalanan ini dunia yang susah ini ketika nabi mengajar muadzam tadi ni alah mandi
CPU times: user 1min 7s, sys: 3.69 s, total: 1min 10s
Wall time: 24.8 s


In [23]:
encoded = transducer_model.encoder_inference(padded_features[0])
g = transducer_model._perform_greedy(encoded, tf.shape(encoded)[0],
                                tf.constant(0, dtype = tf.int32),
                                transducer_model.predict_net.get_initial_state())
g

Hypothesis(index=<tf.Tensor 'while_3/Exit_1:0' shape=() dtype=int32>, prediction=<tf.Tensor 'TensorArrayStack_3/TensorArrayGatherV3:0' shape=(?,) dtype=int32>, states=<tf.Tensor 'while_3/Exit_3:0' shape=(1, 2, 1, 640) dtype=float32>)

In [24]:
indices = g.prediction
minus_one = -1 * tf.ones_like(indices, dtype=tf.int32)
blank_like = 0 * tf.ones_like(indices, dtype=tf.int32)
indices = tf.where(indices == minus_one, blank_like, indices)
num_samples = tf.cast(tf.shape(X[0])[0], dtype=tf.float32)
total_time_reduction_factor = featurizer.frame_step
stime = tf.range(0, num_samples, delta=total_time_reduction_factor, dtype=tf.float32)
stime /= tf.cast(featurizer.sample_rate, dtype=tf.float32)
etime = tf.range(total_time_reduction_factor, num_samples, delta=total_time_reduction_factor, dtype=tf.float32)
etime /= tf.cast(featurizer.sample_rate, dtype=tf.float32)
non_blank = tf.where(tf.not_equal(indices, 0))
non_blank_transcript = tf.gather_nd(indices, non_blank)
non_blank_stime = tf.gather_nd(tf.repeat(tf.expand_dims(stime, axis=-1), tf.shape(indices)[-1], axis=-1), non_blank)[:,0]
non_blank_transcript = tf.identity(non_blank_transcript, name = 'non_blank_transcript')
non_blank_stime = tf.identity(non_blank_stime, name = 'non_blank_stime')

In [25]:
%%time

r = sess.run([non_blank_transcript, non_blank_stime], feed_dict = {X: padded[:1], X_len: lens[:1]})

CPU times: user 8.49 s, sys: 692 ms, total: 9.18 s
Wall time: 4.58 s


In [26]:
list(zip([subwords._id_to_subword(row - 1) for row in r[0]], r[1]))

[('he', 0.010000001), ('lo', 0.020000001)]

In [27]:
saver = tf.train.Saver()
saver.save(sess, 'output-large-conformer/model.ckpt')

'output-large-conformer/model.ckpt'

In [29]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'gather' in n.op.lower()
        or 'placeholder' in n.name
        or 'encoded' in n.name
        or 'decoder' in n.name
        or 'ytu' in n.name
        or 'new_states' in n.name
        or 'padded_' in n.name
        or 'initial_states' in n.name
        or 'non_blank' in n.name)
        and 'adam' not in n.name
        and 'global_step' not in n.name
        and 'Assign' not in n.name
        and 'ReadVariableOp' not in n.name
        and 'Gather' not in n.name
    ]
)
strings.split(',')


['X_placeholder',
 'X_len_placeholder',
 'padded_features',
 'padded_lens',
 'transducer/transducer_prediction/transducer_prediction_embedding/embeddings',
 'greedy_decoder',
 'encoded',
 'encoded_placeholder',
 'predicted_placeholder',
 'states_placeholder',
 'ytu',
 'new_states',
 'initial_states',
 'non_blank_transcript',
 'non_blank_stime']

In [30]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [31]:
freeze_graph('output-large-conformer', strings)

INFO:tensorflow:Restoring parameters from output-large-conformer/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 593 variables.
INFO:tensorflow:Converted 593 variables to const ops.
27225 ops in the final graph.


In [32]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
                
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
        
    return graph

In [33]:
g = load_graph('output-large-conformer/frozen_model.pb')

In [34]:
input_nodes = [
    'X_placeholder',
    'X_len_placeholder',
    'encoded_placeholder',
    'predicted_placeholder',
    'states_placeholder',
]
output_nodes = [
    'greedy_decoder',
    'encoded',
    'ytu',
    'new_states',
    'padded_features',
    'padded_lens',
    'initial_states',
    'non_blank_transcript',
    'non_blank_stime'
]
inputs = {n: g.get_tensor_by_name(f'import/{n}:0') for n in input_nodes}
outputs = {n: g.get_tensor_by_name(f'import/{n}:0') for n in output_nodes}

In [35]:
test_sess = tf.Session(graph = g)

In [36]:
r = test_sess.run(outputs['greedy_decoder'], feed_dict = {inputs['X_placeholder']: padded, 
                                                          inputs['X_len_placeholder']: lens})

In [37]:
for row in r:
    print(malaya_speech.subword.decode(subwords, row[row > 0]))

helo nama saya sen saya tak suka mandi kejap saya masam
helo nama saya using saya suka mandi saya mandi titia
ini dan melalui kenyataan mesej itu mastura menegaskan
pilihan tepat apabila dia kini lebih berani dan
testing nama saya using bin zulkifli
sebut perkataan uncle
tolong sebut nanti kata
nama saya syafiqah idayu
jadi dalam perjalanan ini dunia yang susah ini ketika nabi mengajar muaz bin jabal tadi ni alah ni


In [38]:
encoded_, padded_lens_, s  = test_sess.run([outputs['encoded'], outputs['padded_lens'], outputs['initial_states']], 
                                        feed_dict = {inputs['X_placeholder']: padded, 
                                                     inputs['X_len_placeholder']: lens})

padded_lens_ = padded_lens_ // conformer_model.conv_subsampling.time_reduction_factor

In [39]:
i = 0
r = transducer(
    enc = encoded_[i],
    total = padded_lens_[i],
    initial_states = s,
    encoded_placeholder = inputs['encoded_placeholder'],
    predicted_placeholder = inputs['predicted_placeholder'],
    states_placeholder = inputs['states_placeholder'],
    ytu = outputs['ytu'],
    new_states = outputs['new_states'],
    sess = test_sess,
    beam_width = 1,
)

malaya_speech.subword.decode(subwords, r)

'helo nama saya sen saya tak suka mandi kejap saya masam'

In [40]:
from tensorflow.tools.graph_transforms import TransformGraph

In [41]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

pb = 'output-large-conformer/frozen_model.pb'

input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

transformed_graph_def = TransformGraph(input_graph_def, 
                                           input_nodes,
                                           output_nodes, transforms)
    
with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.gfile.GFile.


In [42]:
g = load_graph('output-large-conformer/frozen_model.pb.quantized')

In [43]:
inputs = {n: g.get_tensor_by_name(f'import/{n}:0') for n in input_nodes}
outputs = {n: g.get_tensor_by_name(f'import/{n}:0') for n in output_nodes}
test_sess = tf.Session(graph = g)

In [44]:
r = test_sess.run(outputs['greedy_decoder'], feed_dict = {inputs['X_placeholder']: padded, 
                                                          inputs['X_len_placeholder']: lens})

In [45]:
for row in r:
    print(malaya_speech.subword.decode(subwords, row[row > 0]))

helo nama saya sen saya tak suka mandi kejap saya masam
helo nama saya using saya suka mandi saya mandi titia
ini dan melalui kenyataan mesej itu mastura menegaskan
pilihan tepat apabila dia kini lebih berani dan
testing nama saya using been zulkifli
sebut perkataan uncle
tolong sebut nanti kata
nama saya syafiqah idayu
jadi dalam perjalanan ini dunia yang susah ini ketika nabi mengajar muaz bin jabal tadi ni alah ni


In [46]:
encoded_, padded_lens_, s  = test_sess.run([outputs['encoded'], outputs['padded_lens'], outputs['initial_states']], 
                                        feed_dict = {inputs['X_placeholder']: padded, 
                                                     inputs['X_len_placeholder']: lens})

padded_lens_ = padded_lens_ // conformer_model.conv_subsampling.time_reduction_factor

In [47]:
i = 0
r = transducer(
    enc = encoded_[i],
    total = padded_lens_[i],
    initial_states = s,
    encoded_placeholder = inputs['encoded_placeholder'],
    predicted_placeholder = inputs['predicted_placeholder'],
    states_placeholder = inputs['states_placeholder'],
    ytu = outputs['ytu'],
    new_states = outputs['new_states'],
    sess = test_sess,
    beam_width = 1,
)

malaya_speech.subword.decode(subwords, r)

'helo nama saya sen saya tak suka mandi kejap saya masam'